# BCS Testing

In [1]:
# imports
import numpy as np
from scipy.stats import qmc

import PyUQTk.uqtkarray as uqtkarray
import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools

import bcs as bcs

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 3
ndim = 2
param = 6 # quad points per dimension for full, or level for sparse
pc_type="LU"
quad_type='sparse'
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs

## Initialize PC

In [3]:
pc_model = uqtkpce.PCSet("NISPnoq", nord, ndim, pc_type, pc_alpha, pc_beta)

## Training points

In [4]:
if False:
# quad points
    pc_model.SetQuadRule(pc_type, quad_type, param)
    samplepts, totquat= pce_tools.UQTkGetQuadPoints(pc_model)
if True:
# random LHS
    npce=pc_model.GetNumberPCTerms()
    nTest=npce
    rng = qmc.LatinHypercube(d=ndim, seed=42)
    samplepts=2*rng.random(n=nTest)-1

## BSC Inputs

In [5]:
sigma = np.array([1.e-08]) # inital noise variance; updated in BCS
eta = 1e-8                           # threshold for stopping the algorithm
lambda_init = np.array([]) # set lambda to a fixed nonnegative value

scale = 0.1     # diagonal loading parameter
adaptive = 1    # generative basis for adaptive CS, 0 or 1
optimal = 1     # use the rigorous implementation of adaptive CS, 0 or 1
verbose = 0     # print statements, 0 or 1

## Define coefficients

In [6]:
npce=pc_model.GetNumberPCTerms()
coef=np.array([i+1 for i in range(npce)])

f_evals=pce_tools.UQTkEvaluatePCE(pc_model, coef, samplepts)

In [7]:
c_k=pce_tools.UQTkBCS(pc_model, f_evals, samplepts, sigma, eta, lambda_init, adaptive, optimal, scale)

Iteration # 0
Iteration # 10


In [8]:
# What goes into the BCS function
# pce_tools.UQTkBSC()

#make quadpoints into uqtk
#x=uqtkarray.dblArray2D()
#x=uqtkarray.numpy2uqtk(np.asfortranarray(qdpts))

#eval basis at the quad points
#Phi = uqtkarray.dblArray2D()
#pc_model.EvalBasisAtCustPts(x,Phi)

#make uqtkarray for evals
#y=uqtkarray.dblArray2D()
#y = uqtkarray.numpy2uqtk(np.asfortranarray(evals))

# create output arrays
#weights = uqtkarray.dblArray1D()      # sparse weights
#used = uqtkarray.intArray1D()         # position of the sparse weights
#errbars = uqtkarray.dblArray1D()      # one standard deviation around the sparse weights
#basis = uqtkarray.dblArray1D()        # if adaptive==1, basis = next projection vector
#alpha = uqtkarray.dblArray1D()        # sparse hyperparameters (1/gamma)
#_lambda = uqtkarray.dblArray1D(1,0.0) # parameter controlling the sparsity

#bcs.BCS(Phi,y,sigma,eta,lambda_init,adaptive,optimal,scale,verbose,weights,used,errbars,basis,alpha,_lambda)

#coef2=np.zeros(npce)
#for i in range(npce):
 #   pos = used[i]
  #  coef2[pos]=weights[i]

In [9]:
import pandas as pd
pd.DataFrame(data=np.transpose([c_k, coef]), columns=['bcs', 'actual'])

,bcs,actual
0,1.000000,1.0
1,1.999999,2.0
2,2.999999,3.0
3,4.000000,4.0
4,5.000000,5.0
5,6.000000,6.0
6,6.999999,7.0
7,7.999997,8.0
8,8.999997,9.0
9,9.999999,10.0
